# Argumentation Abstraite (Dung)

Ce notebook couvre les cadres d'argumentation abstraits de Dung.

**Pre-requis**: Executez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement.

**Navigation**: [Precedent](Tweety-4-Belief-Revision.ipynb) | [Suivant](Tweety-6-Structured-Argumentation.ipynb)


In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import sys
import pathlib
import shutil
import platform

# === Configuration des outils externes (standalone) ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    # Also check if it's a directory (for Tweety compatibility)
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# Auto-detection + Auto-installation Clingo (binaire)
# IMPORTANT: TweetyProject ClingoSolver expects a DIRECTORY path, not the full executable path.
# It appends "/clingo" (Unix) or "/clingo.exe" (Windows) internally when invoking the solver.
clingo_paths = [
    shutil.which("clingo"),
    shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/clingo/clingo.exe"),
    pathlib.Path("../ext_tools/clingo/clingo.exe"),
]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str):
            # For system PATH clingo, use the directory containing it
            cp_path = pathlib.Path(cp)
            EXTERNAL_TOOLS["CLINGO"] = str(cp_path.parent.resolve())
            break
        elif cp.exists():
            # For local clingo.exe, store the DIRECTORY path (parent)
            EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve())
            break

# Auto-install clingo if not found (Windows only)
if not EXTERNAL_TOOLS.get("CLINGO") and platform.system() == "Windows":
    print("\nClingo non trouve. Installation automatique...")
    try:
        import subprocess
        install_script = pathlib.Path("install_clingo.py")
        if not install_script.exists():
            install_script = pathlib.Path("Argument_Analysis/install_clingo.py")

        if install_script.exists():
            result = subprocess.run(
                [sys.executable, str(install_script), "--target-dir", "ext_tools/clingo"],
                capture_output=True, text=True, timeout=300
            )
            if result.returncode == 0:
                print("Installation de Clingo reussie!")
                # Recheck after installation
                clingo_exe = pathlib.Path("ext_tools/clingo/clingo.exe")
                if clingo_exe.exists():
                    EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.parent.resolve())
            else:
                print(f"Echec installation Clingo: {result.stderr}")
        else:
            print("Script d'installation install_clingo.py non trouve.")
    except Exception as e:
        print(f"Erreur lors de l'auto-installation de Clingo: {e}")

# Auto-detection SPASS
spass_paths = [
    shutil.which("SPASS"),
    shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/spass/SPASS.exe"),
    pathlib.Path("../ext_tools/spass/SPASS.exe"),
]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str):
            EXTERNAL_TOOLS["SPASS"] = sp
            break
        elif sp.exists():
            EXTERNAL_TOOLS["SPASS"] = str(sp.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists():
        jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
        print("JVM non disponible.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools:
        print(f"Outils externes: {', '.join(tools)}")

## Partie 4 : Argumentation Abstraite et Structurée
<a id="partie4"></a>

Nous entrons maintenant au cœur de l'argumentation computationnelle, en commençant par le cadre fondateur de Dung et en progressant vers des approches qui prennent en compte la structure interne des arguments. Cette partie est souvent plus stable car elle repose sur des modules centraux de Tweety.

### 4.1 Cadres d'Argumentation Abstraits (Dung)
<a id="4.1"></a>

Le cadre de Dung (Abstract Argumentation Framework - AAF) est la base de nombreuses extensions. Il modélise les conflits entre arguments de manière abstraite.

* Un ensemble d'**Arguments** (`Argument`) considérés comme des entités atomiques.
* Une relation d'**Attaque** (`Attack`) binaire entre arguments ($a$ attaque $b$).
* L'ensemble forme une **Théorie de Dung** (`DungTheory`).

L'objectif est de déterminer des ensembles d'arguments collectivement acceptables, appelés **extensions**, selon différentes **sémantiques** (Conflict-Free, Admissible, Complete, Grounded, Preferred, Stable, etc.). Tweety fournit des **raisonneurs** (`org.tweetyproject.arg.dung.reasoner.*`) pour calculer ces extensions.

* **Conflict-Free (CF)**: Aucun argument dans l'extension n'attaque un autre argument de l'extension.
* **Admissible (ADM)**: Conflict-Free + chaque argument de l'extension est défendu par l'extension contre ses attaquants.
* **Complete (COM)**: Admissible + contient tous les arguments qu'elle défend.
* **Grounded (GR)**: L'extension complète minimale (calculée itérativement, sceptique).
* **Preferred (PRF)**: Une extension complète maximale (par inclusion). Il peut y en avoir plusieurs (crédule).
* **Stable (ST)**: Conflict-Free + attaque tous les arguments qui ne sont pas dans l'extension. Peut ne pas exister, mais si elle existe, elle est aussi Preferred et Complete.
* **CF2**: Sémantique gérant les cycles impairs via la décomposition en Composantes Fortement Connexes (SCC).

Tweety fournit des **raisonneurs** (`org.tweetyproject.arg.dung.reasoner.*`) pour calculer ces extensions (ex: `SimpleGroundedReasoner`, `SimpleStableReasoner`, `SccCF2Reasoner`).

In [ ]:
# --- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
print("\n--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple Dung...")
    try:
        # Imports nécessaires pour Dung
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        # Importer plusieurs raisonneurs
        from org.tweetyproject.arg.dung.reasoner import (
            SimpleGroundedReasoner, SimpleStableReasoner, SimplePreferredReasoner,
            SimpleCompleteReasoner, SimpleAdmissibleReasoner, SimpleConflictFreeReasoner
        )
        # Importer Semantics si besoin pour certains raisonneurs plus avancés (non utilisés ici)
        # from org.tweetyproject.arg.dung.semantics import Semantics
        from java.util import Collection # Pour vérifier taille retour

        print("✔️ Imports Dung réussis.")

        # --- Exemple 1: A <-> B -> C ---
        af1 = DungTheory()
        a1 = Argument("a1"); b1 = Argument("b1"); c1 = Argument("c1")
        # Ajouter des arguments
        af1.add(a1); af1.add(b1); af1.add(c1)
        # Ajouter des attaques
        af1.add(Attack(a1, b1)); af1.add(Attack(b1, a1)); af1.add(Attack(b1, c1))

        print("\n--- AF1: a1 <-> b1 -> c1 ---")
        print("Cadre :", af1)

        # Calculer les extensions pour différentes sémantiques
        print("\nCalcul des extensions pour AF1:")
        try:
            # Grounded: getModel() retourne une Extension, getModels() retourne une Collection<Extension>
            grounded_ext = SimpleGroundedReasoner().getModel(af1) # Plus direct pour grounded
            print(f" - Grounded   : {{{grounded_ext}}}") # Afficher comme un ensemble pour la cohérence
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts = SimpleStableReasoner().getModels(af1)
            print(f" - Stable     ({stable_exts.size()}):", stable_exts)
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts = SimplePreferredReasoner().getModels(af1)
            print(f" - Preferred  ({preferred_exts.size()}):", preferred_exts)
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts = SimpleCompleteReasoner().getModels(af1)
            print(f" - Complete   ({complete_exts.size()}):", complete_exts)
        except Exception as e: print(f"   Erreur Complete: {e}")
        try:
            admissible_exts = SimpleAdmissibleReasoner().getModels(af1)
            # Convertir la Collection Java en liste Python pour len()
            # Note: Cela peut être coûteux si la collection est énorme
            admissible_list = list(admissible_exts)
            print(f" - Admissible ({len(admissible_list)}):", admissible_exts) # Afficher la collection Java
        except Exception as e: print(f"   Erreur Admissible: {e}")
        try:
            conflict_free_exts = SimpleConflictFreeReasoner().getModels(af1)
            cf_list = list(conflict_free_exts)
            print(f" - ConflictFree ({len(cf_list)}):", conflict_free_exts) # Attention, peut être très grand
        except Exception as e: print(f"   Erreur ConflictFree: {e}")


        # --- Exemple 2: Cycle a->b->c->a ---
        af_cycle = DungTheory()
        a_cy = Argument("a_cy"); b_cy = Argument("b_cy"); c_cy = Argument("c_cy")
        af_cycle.add(a_cy); af_cycle.add(b_cy); af_cycle.add(c_cy)
        af_cycle.add(Attack(a_cy, b_cy)); af_cycle.add(Attack(b_cy, c_cy)); af_cycle.add(Attack(c_cy, a_cy))

        print("\n--- AF Cycle: a -> b -> c -> a ---")
        print("Cadre :", af_cycle)
        print("\nCalcul des extensions pour AF Cycle:")
        try:
            print(" - Grounded   :", SimpleGroundedReasoner().getModel(af_cycle)) # {}
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts_cy = SimpleStableReasoner().getModels(af_cycle)
            print(f" - Stable     ({stable_exts_cy.size()}):", stable_exts_cy) # {}
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts_cy = SimplePreferredReasoner().getModels(af_cycle)
            print(f" - Preferred  ({preferred_exts_cy.size()}):", preferred_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts_cy = SimpleCompleteReasoner().getModels(af_cycle)
            print(f" - Complete   ({complete_exts_cy.size()}):", complete_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Complete: {e}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour l'Argumentation Abstraite : {e}")
        print("   Vérifiez le JAR 'org.tweetyproject.arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple Dung: {e_java.message()}")
        # print(e_java.stacktrace()) # Décommenter pour trace Java complète
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple Dung: {e_gen}")
        import traceback
        traceback.print_exc()

#### 4.1.2 Sémantique CF2
<a id="4.1.2"></a>

La sémantique CF2 ([Baroni, Giacomin, Guida, *SCC-recursive semantics for abstract argumentation frameworks*, 2005](https://link.springer.com/chapter/10.1007/11557791_3)) est une alternative aux sémantiques classiques qui gère différemment les cycles, notamment les cycles impairs. Elle se base sur une décomposition du graphe d'argumentation en Composantes Fortement Connexes (SCCs). Tweety fournit le `SccCF2Reasoner`.

In [ ]:
# --- 4.1.2 Sémantique CF2 ---
print("\n--- 4.1.2 Sémantique CF2 ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple CF2...")
    try:
        # Imports (peuvent être déjà faits, mais sécurité)
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import SccCF2Reasoner
        from org.tweetyproject.arg.dung.semantics import Extension # Pour type hinting éventuel
        from java.util import Collection

        print("✔️ Imports Dung (pour CF2) réussis.")

        # --- Exemple AF2 : Cycle a->b->c->d->e->a, e->f ---
        # (Recréation pour autonomie de la cellule)
        af2 = DungTheory()
        args_cf2_map = {name: Argument(name) for name in "abcdef"} # Utiliser un dictionnaire
        for arg in args_cf2_map.values(): af2.add(arg)

        attacks_cf2 = [("a","b"), ("b","c"), ("c","d"), ("d","e"), ("e","a"), ("e","f")]
        for s, t in attacks_cf2:
             # S'assurer que les arguments existent avant d'ajouter l'attaque
             if s in args_cf2_map and t in args_cf2_map:
                  af2.add(Attack(args_cf2_map[s], args_cf2_map[t]))
             else:
                  print(f"WARN: Argument(s) non trouvé(s) pour l'attaque ({s},{t})")

        print("\n--- AF pour CF2 : Cycle a->b->c->d->e->a, e->f ---")
        print("Cadre :", af2)

        # --- Raisonnement CF2 ---
        cf2_reasoner = SccCF2Reasoner()
        print("\nCalcul des extensions CF2:")
        try:
            cf2_extensions_collection = cf2_reasoner.getModels(af2) # Retourne Collection<Extension>
            if cf2_extensions_collection.isEmpty():
                 print("  (Aucune extension CF2 trouvée)")
            else:
                 # Itérer sur la collection Java
                 ext_iterator = cf2_extensions_collection.iterator()
                 count = 0
                 while ext_iterator.hasNext():
                      ext = ext_iterator.next()
                      # Pour un affichage plus propre des arguments dans l'extension:
                      args_in_ext = ", ".join(sorted([str(arg.getName()) for arg in ext]))
                      print(f"  - {{{args_in_ext}}}")
                      count += 1
                 print(f"  ({count} extension(s) trouvée(s))")

        except jpype.JException as e_cf2_java:
             print(f"  ❌ Erreur Java lors du raisonnement CF2: {e_cf2_java.message()}")
             # print(e_cf2_java.stacktrace())
        except Exception as e_cf2_py:
              print(f"  ❌ Erreur Python lors du raisonnement CF2: {e_cf2_py}")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour CF2 : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple CF2: {e_java.message()}")
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple CF2: {e_gen}")
        import traceback
        traceback.print_exc()

#### 4.1.3 Génération de Cadres d'Argumentation
<a id="4.1.3"></a>

Tweety fournit des outils pour générer aléatoirement des cadres d'argumentation abstraits (`DungTheory`), ce qui est utile pour les tests, benchmarks ou simulations. `DefaultDungTheoryGenerator` est un générateur simple paramétrable.

In [ ]:
# --- 4.1.3 Génération de Cadres Dung ---
print("\n--- 4.1.3 Génération de Cadres Dung ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple de génération...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.util import DefaultDungTheoryGenerator, DungTheoryGenerationParameters

        print("✔️ Imports pour génération Dung réussis.")

        # Paramètres de génération
        params = DungTheoryGenerationParameters()
        params.numberOfArguments = 6
        params.attackProbability = 0.25
        # CORRECTION: La ligne suivante est retirée car le champ n'existe probablement plus
        # params.allowSelfAttacks = False
        print(f"ℹ️ Paramètres de génération: {params.numberOfArguments} args, proba_attaque={params.attackProbability} (comportement par défaut pour auto-attaques)")

        generator = DefaultDungTheoryGenerator(params)

        # Générer UN cadre
        generated_af = generator.next()

        print("\nCadre généré aléatoirement :")
        print(f"  Arguments: {generated_af.getNodes()}")
        print(f"  Attaques : {generated_af.getAttacks()}")
        print(f"  Représentation compacte: {generated_af}")

    except ImportError as e:
        print(f"❌ Erreur d'import pour la génération Dung : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple génération Dung: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple génération Dung: {e_gen}")
        import traceback
        traceback.print_exc()

#### 4.1.4 Apprentissage de Cadres d'Argumentation (depuis Labellisations)
<a id="4.1.4"></a>

L'apprentissage de cadre vise à reconstruire un cadre d'argumentation (ou un ensemble de cadres possibles) à partir d'informations partielles, typiquement des exemples de labellisations d'arguments (IN, OUT, UNDEC) selon une certaine sémantique. Tweety propose `SimpleAFLearner` pour cela.

In [ ]:
# --- 4.1.4 Apprentissage de Cadres Dung (depuis Labellisations) ---
print("\n--- 4.1.4 Apprentissage de Cadres Dung ---")

print("!! SECTION COMMENTÉE !!")
print("L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety")
print("(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling")
print("pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.")

# if not jvm_ready:
#     print("❌ ERREUR: JVM non démarrée.")
# else:
#     print("ℹ️ JVM prête. Exécution de l'exemple d'apprentissage...")
#     try:
#         # Imports
#         import jpype
#         from jpype.types import *
#         from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
#         from org.tweetyproject.arg.dung.learning import SimpleAFLearner
#         from org.tweetyproject.arg.dung.learning.syntax import Entity
#         from org.tweetyproject.arg.dung.semantics import Semantics
#         # Import de Label retiré (correction précédente)
#
#         print("✔️ Imports pour apprentissage Dung réussis.")
#
#         # 1. Définir le cadre "caché"
#         hidden_af = DungTheory()
#         a_learn = Argument("a"); b_learn = Argument("b"); c_learn = Argument("c")
#         hidden_af.add(a_learn); hidden_af.add(b_learn); hidden_af.add(c_learn)
#         hidden_af.add(Attack(a_learn, b_learn)); hidden_af.add(Attack(b_learn, a_learn)); hidden_af.add(Attack(b_learn, c_learn))
#         print("\nCadre caché (à apprendre):", hidden_af)
#
#         # 2. Créer l'Oracle
#         oracle = Entity(hidden_af)
#         arguments_known = oracle.getArguments()
#         print("Arguments connus par l'apprenant:", arguments_known)
#
#         # 3. Créer l'Apprenant
#         learner = SimpleAFLearner(arguments_known)
#         print(f"État initial apprenant: {learner.getNumberOfFrameworks()} cadres compatibles possibles.")
#
#         # 4. Apprendre depuis des labellisations
#         # Labellisation Stable
#         print("\nApprentissage depuis labellisation Stable:")
#         try:
#              labeling_stable = oracle.getLabeling(Semantics.STABLE_SEMANTICS) # C'est ici que l'erreur se produit
#              print(f"  Oracle fournit Labellisation STABLE: {labeling_stable}")
#              learner.learnLabeling(labeling_stable)
#              print(f"  Après STABLE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_stable:
#               print(f"  Erreur lors de l'obtention/apprentissage de la labellisation Stable: {e_stable.message()}")
#               # Afficher la stacktrace Java peut aider à confirmer l'erreur interne
#               # print(e_stable.stacktrace())
#
#         # Labellisation Conflict-Free (pourrait aussi échouer)
#         print("\nApprentissage depuis labellisation Conflict-Free:")
#         try:
#              labeling_cf = oracle.getLabeling(Semantics.CONFLICTFREE_SEMANTICS)
#              print(f"  Oracle fournit Labellisation CONFLICT_FREE: {labeling_cf}")
#              learner.learnLabeling(labeling_cf)
#              print(f"  Après CONFLICT_FREE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_cf:
#              print(f"  Erreur lors de l'obtention/apprentissage de la labellisation ConflictFree: {e_cf.message()}")
#
#
#         # 5. Obtenir le(s) cadre(s) appris
#         # Ce code ne sera probablement pas atteint
#         learned_af = learner.getModel()
#         print("\nCadre appris (un des cadres compatibles) :")
#         print(str(learned_af))
#
#     # ... (Blocs except ImportError, JException, Exception comme avant) ...
#     except ImportError as e:
#        print(f"❌ Erreur d'import pour l'apprentissage Dung : {e}.")
#     except jpype.JException as e_java:
#        print(f"❌ Erreur Java générale dans l'exemple apprentissage Dung: {e_java.message()}")
#     except Exception as e_gen:
#        print(f"❌ Erreur Python inattendue dans l'exemple apprentissage Dung: {e_gen}")
#        import traceback
#        traceback.print_exc()

---

## Navigation

- **Precedent**: [Tweety-4-Belief-Revision.ipynb](Tweety-4-Belief-Revision.ipynb)
- **Suivant**: [Tweety-6-Structured-Argumentation.ipynb](Tweety-6-Structured-Argumentation.ipynb)
- **Index**: [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb)
